<a href="https://colab.research.google.com/github/cpuhrsch/maskedtensor/blob/main/notebooks/issue67180.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import numpy as np
if torch.__version__ != "1.11.0.dev20210927+cpu":
    !pip uninstall --y torch
    !pip install https://download.pytorch.org/whl/nightly/cpu/torch-1.11.0.dev20210927%2Bcpu-cp37-cp37m-linux_x86_64.whl
    raise RuntimeError("Restart runtime now")

In [2]:
!pip install git+https://github.com/cpuhrsch/maskedtensor

  Cloning https://github.com/cpuhrsch/maskedtensor to /tmp/pip-req-build-9f32hu0h
  Running command git clone -q https://github.com/cpuhrsch/maskedtensor /tmp/pip-req-build-9f32hu0h


In [3]:
# Import factory function
from maskedtensor import masked_tensor
from maskedtensor import as_masked_tensor

In [4]:
y = torch.arange(32).float()
x = y * y.fmod(4)
x = x.masked_fill(x == 0, float('nan'))
print(x)
print(torch.nanmean(x))
print(torch.mean(masked_tensor(x, ~torch.isnan(x))))

tensor([nan,  1.,  4.,  9., nan,  5., 12., 21., nan,  9., 20., 33., nan, 13.,
        28., 45., nan, 17., 36., 57., nan, 21., 44., 69., nan, 25., 52., 81.,
        nan, 29., 60., 93.])
tensor(32.6667)
masked_tensor( 32.6667, True)


[Issue 67180](https://github.com/pytorch/pytorch/issues/67180) encounters a long standing issue around nan gradients that stem from the fact that PyTorch can't differentiate between zero and nan gradients. MaskedTensor could be used to resolve this in one way by masking out the nan in the forward pass isntead of using nansum.

In [5]:
a = torch.tensor([1., 2., float('nan')])
b = torch.tensor(1.0, requires_grad=True)
c = a * b
c1 = torch.nansum(c)  # or torch.nanmean

bgrad1, = torch.autograd.grad(c1, b, retain_graph=True)
print(bgrad1)

tensor(nan)


In [6]:
a = torch.tensor([1., 2., float('nan')])
b = torch.tensor(1.0, requires_grad=True)
ma = masked_tensor(a, ~torch.isnan(a))
c = ma * b
c1 = torch.sum(c)  # or torch.nanmean

bgrad1, = torch.autograd.grad(c1, b, retain_graph=True)
print(ma)
print(b)
print(c)
print(c1)
print(bgrad1)

masked_tensor(
  [  1.0000,   2.0000,       --]
)
tensor(1., requires_grad=True)
masked_tensor(
  [  1.0000,   2.0000,       --]
)
masked_tensor(  3.0000, True)
masked_tensor(  3.0000, True)
